## Основные импорты

In [4]:
from lyricsgenius import Genius
from api_key import *
from time import sleep
import re
import pandas as pd

In [2]:
#в одной песне есть запятая, поэтому разделитель заменяется
df = pd.read_csv("data/interpol.csv")
df.head()

,Unnamed: 0,Artist,Album Name,Track Name,Key,Tempo,Mode
0,0,Interpol,Turn On the Bright Lights,Untitled,Ab,92,Major
1,1,Interpol,Turn On the Bright Lights,Obstacle 1,F,121,Major
2,2,Interpol,Turn On the Bright Lights,NYC,F,149,Major
3,3,Interpol,Turn On the Bright Lights,PDA,F#,138,Minor
4,4,Interpol,Turn On the Bright Lights,Say Hello to the Angels,A,103,Minor


In [3]:
#если появилась колонка "Unnamed: 0"
df = df.drop("Unnamed: 0", axis = 1)
df.head()

,Artist,Album Name,Track Name,Key,Tempo,Mode
0,Interpol,Turn On the Bright Lights,Untitled,Ab,92,Major
1,Interpol,Turn On the Bright Lights,Obstacle 1,F,121,Major
2,Interpol,Turn On the Bright Lights,NYC,F,149,Major
3,Interpol,Turn On the Bright Lights,PDA,F#,138,Minor
4,Interpol,Turn On the Bright Lights,Say Hello to the Angels,A,103,Minor


## Получение текста песен, экспорт в новый файл

In [4]:
token = CLIENT_ACCESS_TOKEN
genius = Genius(token, retries=3) #три попытки при поиске песни, на всякий случай
df['Lyrics'] = None

In [5]:
#поиск песенки
def find_lyrics(row):
    artist = row['Artist']
    song_name = row['Track Name']
    
    try:
        song = genius.search_song(song_name, artist)
        if song:
            lyrics = song.lyrics
            return lyrics
        else:
            print(f"Не найден текст для песни {song_name}.")
            return "None"
    
    except Exception as e:
        print(f"Произошла ошибка для {song_name}: {e}")
    
    finally:
        sleep(0.5)

In [19]:
#применить ко всем песенкам
df['Lyrics'] = df.apply(find_lyrics, axis=1)

df.to_json("data/interpol.json", index=False)

Searching for "Untitled" by Interpol...


/home/faust/.local/lib/python3.12/site-packages/lyricsgenius/genius.py:468: FutureWarning: The constructor signature will change in a future version. It will change to Song(lyrics, body) instead of Song(client, json_dict, lyrics).
  song = Song(self, song_info, lyrics)


Done.
Searching for "Obstacle 1" by Interpol...
Done.
Searching for "NYC" by Interpol...
Done.
Searching for "PDA" by Interpol...
Done.
Searching for "Say Hello to the Angels" by Interpol...
Done.
Searching for "Hands Away" by Interpol...
Done.
Searching for "Obstacle 2" by Interpol...
Done.
Searching for "Stella Was a Diver and She Was Always Down" by Interpol...
Done.
Searching for "Roland" by Interpol...
Done.
Searching for "The New" by Interpol...
Done.
Searching for "Leif Erikson" by Interpol...
Done.
Searching for "Next Exit" by Interpol...
Done.
Searching for "Evil" by Interpol...
Done.
Searching for "Narc" by Interpol...
Done.
Searching for "Take You on a Cruise" by Interpol...
Done.
Searching for "Slow Hands" by Interpol...
Done.
Searching for "Not Even Jail" by Interpol...
Done.
Searching for "Public Pervert" by Interpol...
Done.
Searching for "C'mere" by Interpol...
Done.
Searching for "Length of Love" by Interpol...
Done.
Searching for "A Time to Be So Small" by Interpol...

## Чистка текста песен, разделение

In [10]:
new_df = pd.read_json("data/interpol.json")
new_df.head()

,Artist,Album Name,Track Name,Key,Tempo,Mode,Lyrics Cleaned
0,Interpol,Turn On the Bright Lights,Untitled,Ab,92,Major,"[[Surprise, sometimes, will come around, Surpr..."
1,Interpol,Turn On the Bright Lights,Obstacle 1,F,121,Major,[[I wish I could eat the salt off of your lost...
2,Interpol,Turn On the Bright Lights,NYC,F,149,Major,"[[I had seven faces, Thought I knew which one ..."
3,Interpol,Turn On the Bright Lights,PDA,F#,138,Minor,"[[Yours is the only version of my desertion, T..."
4,Interpol,Turn On the Bright Lights,Say Hello to the Angels,A,103,Minor,"[[I want your silent parts, the parts the bird..."


In [6]:
lyric = new_df.iloc[3]['Lyrics']
print(lyric)

KeyError: 'Lyrics'

In [25]:
def clean_lyrics(lyric):
    if not isinstance(lyric, str):
        return lyric
    
    #удаляем пояснения до текста песни (до первой квадратной скобочки)
    new_start_index = lyric.find("[")
    if new_start_index != -1:
        lyric = lyric [new_start_index:]

    lines = lyric.split('\n')

    result = []
    song_section = []

    for line in lines:
        #если есть обозначение раздела типо куплета, припева
        if re.match(r'\[.*?\]', line.strip()):
            #сохраение раздела и начало нового
            if song_section:
                result.append(song_section)
                song_section = []
        #если строка, которая не название раздела
        elif line.strip():
            song_section.append(line.strip())
    
    if song_section:
        result.append(song_section)
    return result

In [2]:
new_df['Lyrics Cleaned'] = new_df['Lyrics'].apply(clean_lyrics)
new_df.head()

NameError: name 'new_df' is not defined

In [11]:
#проблемный текст песни
lyric = new_df['Lyrics Cleaned'][5]
lyric

[['Will you put my hands away?',
  'Will you be my man?',
  "Serve it up, don't wait",
  "Let's see about this ham",
  'Oh, what happened?',
  'Oh, what happened?',
  "Home spun desperation's knowing",
  "Inside your cover's always blown"]]

In [28]:
#удаление старого текста
new_df = new_df.drop('Lyrics', axis=1)

new_df.head()

,Artist,Album Name,Track Name,Key,Tempo,Mode,Lyrics Cleaned
0,Interpol,Turn On the Bright Lights,Untitled,Ab,92,Major,"[[Surprise, sometimes, will come around, Surpr..."
1,Interpol,Turn On the Bright Lights,Obstacle 1,F,121,Major,[[I wish I could eat the salt off of your lost...
2,Interpol,Turn On the Bright Lights,NYC,F,149,Major,"[[I had seven faces, Thought I knew which one ..."
3,Interpol,Turn On the Bright Lights,PDA,F#,138,Minor,"[[Yours is the only version of my desertion, T..."
4,Interpol,Turn On the Bright Lights,Say Hello to the Angels,A,103,Minor,"[[I want your silent parts, the parts the bird..."


In [29]:
new_df.to_json("data/interpol.json")